In [1]:
import pandas as pd

file_path = "C:\\Users\\LENOVO\\Desktop\\Crediable assessment\\BI_Analyst_Case_Study_Data.xlsx"

xls = pd.ExcelFile(file_path, engine="openpyxl")  # Explicitly specify openpyxl
print(xls.sheet_names)

['Disbursements', 'Repayments']


In [26]:
# Load the data from both sheets
disbursements_df = pd.read_excel(xls, sheet_name="Disbursements")
repayments_df = pd.read_excel(xls, sheet_name="Repayments")

# Display first few rows of each dataframe
disbursements_df.head(), repayments_df.head()


(                                         customer_id  disb_date   tenure  \
 0  91810ca1aa097db79f050f38e9946fa5482b4e28c925e2... 2024-03-19  14 days   
 1  42ca06e6fe1ff9803e82a5c20184671b54090e488f78d6... 2024-03-19   7 days   
 2  b23747f53af805e18ad16a4ef235b6642d88f9134644ff... 2024-03-19   7 days   
 3  1bd32f9b083fc6ddfffd65730fbfa66654fa76a19b0b0e... 2024-03-19  14 days   
 4  e7cfbaa97ba7702c52df5f1dddba54bd26923ebad945f1... 2024-03-19   7 days   
 
                         account_num  loan_amount  loan_fee  
 0  3O66YENWELA6E2H1R9YLX0LDZNOMNHD4          360      43.2  
 1  6XWHXKKR1W2HIA8I0V75PZFZBXUUGSVO           70       7.0  
 2  OCGK3RJZ91A999VXD4VB3LATPSME3J5L         3500     350.0  
 3  9X3Q682DOR7927IMMJLFHBGP0RP7YF5C         3500     420.0  
 4  AQH88NNF8S76MGJL4J4ULEAE18O0KLWH          120      12.0  ,
                          date_time  \
 0  27-JUN-24 07.16.36.000000000 AM   
 1  27-JUN-24 05.26.50.000000000 PM   
 2  27-JUN-24 06.45.40.000000000 PM   
 3  27-

In [9]:
import pandas as pd
import sqlite3

In [10]:
# Load sheets into DataFrames
disbursements_df = pd.read_excel(xls, sheet_name="Disbursements")
repayments_df = pd.read_excel(xls, sheet_name="Repayments")

# Create SQLite in-memory database
conn = sqlite3.connect(":memory:")

# Save data to SQL tables
disbursements_df.to_sql("Disbursements", conn, index=False, if_exists="replace")
repayments_df.to_sql("Repayments", conn, index=False, if_exists="replace")

66016

In [28]:
# Load data into SQL tables
disbursements_df.to_sql("Disbursements", conn, index=False, if_exists="replace")
repayments_df.to_sql("Repayments", conn, index=False, if_exists="replace")

# SQL query to calculate outstanding balances for each customer
query = """
SELECT 
    d.customer_id, 
    d.loan_amount, 
    IFNULL(SUM(r.amount), 0) AS total_repaid, 
    (d.loan_amount - IFNULL(SUM(r.amount), 0)) AS outstanding_balance
FROM Disbursements d
LEFT JOIN Repayments r ON d.customer_id = r.customer_id
GROUP BY d.customer_id, d.loan_amount
ORDER BY outstanding_balance DESC;
"""

# Execute query and convert result to DataFrame
credit_exposure_df = pd.read_sql(query, conn)

# Display the first few rows
print(credit_exposure_df.head())


                                         customer_id  loan_amount  \
0  1cdcc3dd95cdfad6618ceae4d951e4df21b556cd06f3b6...         3500   
1  4c4042e1517ed50bc4f9ac6edb40b00f94873bfb8d8e83...         2240   
2  ee44dc043cfe2b0618854cc7e56833e277fe2b33418c05...          950   
3  443144f728d5a23010817c470717bd54c7f98167eb6e13...         1000   
4  7adb0ef8b2f9cc2f927fc12e6112cfbc15cec7cfb5a4b2...         1030   

   total_repaid  outstanding_balance  
0       1089.86              2410.14  
1        725.00              1515.00  
2        240.98               709.02  
3        484.47               515.53  
4        602.62               427.38  


In [21]:
#Provisioning and Write-Off Analysis
query = """
SELECT 
    d.customer_id, 
    d.loan_amount, 
    (d.loan_amount - IFNULL(SUM(r.amount), 0)) AS outstanding_balance,
    d.disb_date,
    julianday('now') - julianday(d.disb_date) AS days_overdue,
    CASE 
        WHEN julianday('now') - julianday(d.disb_date) BETWEEN 30 AND 60 THEN (d.loan_amount - IFNULL(SUM(r.amount), 0)) * 0.10
        WHEN julianday('now') - julianday(d.disb_date) BETWEEN 61 AND 90 THEN (d.loan_amount - IFNULL(SUM(r.amount), 0)) * 0.25
        WHEN julianday('now') - julianday(d.disb_date) BETWEEN 91 AND 180 THEN (d.loan_amount - IFNULL(SUM(r.amount), 0)) * 0.50
        WHEN julianday('now') - julianday(d.disb_date) > 180 THEN (d.loan_amount - IFNULL(SUM(r.amount), 0)) * 1.00
        ELSE 0
    END AS provision_amount
FROM Disbursements d
LEFT JOIN Repayments r ON d.customer_id = r.customer_id
GROUP BY d.customer_id, d.disb_date;
"""

# Execute query
provisioning_df = pd.read_sql(query, conn)

# Display results
print("\nProvisioning & Write-Off Analysis:")
print(provisioning_df.head())



Provisioning & Write-Off Analysis:
                                         customer_id  loan_amount  \
0  000514554c34603e8a7551050e988732cf11a22de40fa6...         3500   
1  000514554c34603e8a7551050e988732cf11a22de40fa6...         3500   
2  000514554c34603e8a7551050e988732cf11a22de40fa6...         3500   
3  000514554c34603e8a7551050e988732cf11a22de40fa6...         3500   
4  000514554c34603e8a7551050e988732cf11a22de40fa6...         3500   

   outstanding_balance            disb_date  days_overdue  provision_amount  
0             -27600.0  2024-01-14 00:00:00    432.555135          -27600.0  
1             -27600.0  2024-02-23 00:00:00    392.555135          -27600.0  
2             -27600.0  2024-03-07 00:00:00    379.555135          -27600.0  
3             -27600.0  2024-04-24 00:00:00    331.555135          -27600.0  
4             -27600.0  2024-05-08 00:00:00    317.555135          -27600.0  


In [23]:
# Portfolio Risk Alerts
query = """
SELECT 
    d.customer_id, 
    d.loan_amount, 
    (d.loan_amount - IFNULL(SUM(r.amount), 0)) AS outstanding_balance,
    d.disb_date,
    julianday('now') - julianday(d.disb_date) AS days_overdue,
    CASE 
        WHEN julianday('now') - julianday(d.disb_date) > 60 THEN 'HIGH RISK'
        WHEN julianday('now') - julianday(d.disb_date) BETWEEN 30 AND 60 THEN 'MEDIUM RISK'
        ELSE 'LOW RISK'
    END AS risk_level
FROM Disbursements d
LEFT JOIN Repayments r ON d.customer_id = r.customer_id
GROUP BY d.customer_id, d.disb_date
ORDER BY risk_level DESC, outstanding_balance DESC;
"""

# Execute query
risk_alerts_df = pd.read_sql(query, conn)

# Display results
print("\nRisk Alerts & Portfolio Monitoring:")
print(risk_alerts_df.head())



Risk Alerts & Portfolio Monitoring:
                                         customer_id  loan_amount  \
0  1cdcc3dd95cdfad6618ceae4d951e4df21b556cd06f3b6...         3500   
1  4c4042e1517ed50bc4f9ac6edb40b00f94873bfb8d8e83...         2240   
2  ee44dc043cfe2b0618854cc7e56833e277fe2b33418c05...          950   
3  443144f728d5a23010817c470717bd54c7f98167eb6e13...         1000   
4  7adb0ef8b2f9cc2f927fc12e6112cfbc15cec7cfb5a4b2...         1030   

   outstanding_balance            disb_date  days_overdue risk_level  
0              2410.14  2024-01-02 00:00:00    444.557451  HIGH RISK  
1              1515.00  2024-02-09 00:00:00    406.557451  HIGH RISK  
2               709.02  2024-02-20 00:00:00    395.557451  HIGH RISK  
3               515.53  2024-03-04 00:00:00    382.557451  HIGH RISK  
4               427.38  2024-02-24 00:00:00    391.557451  HIGH RISK  


In [24]:
#Loan Product Recommendations
query = """
SELECT 
    d.customer_id, 
    d.loan_amount, 
    IFNULL(SUM(r.amount), 0) AS total_repaid, 
    (d.loan_amount - IFNULL(SUM(r.amount), 0)) AS outstanding_balance,
    COUNT(CASE WHEN r.date_time <= d.disb_date THEN 1 ELSE NULL END) AS on_time_payments,
    COUNT(CASE WHEN r.date_time > d.disb_date THEN 1 ELSE NULL END) AS late_payments,
    CASE 
        WHEN COUNT(CASE WHEN r.date_time <= d.disb_date THEN 1 ELSE NULL END) >= 3 THEN 'LOYAL CUSTOMER'
        WHEN COUNT(CASE WHEN r.date_time > d.disb_date THEN 1 ELSE NULL END) >= 3 THEN 'NEEDS FLEXIBLE TERMS'
        ELSE 'STANDARD CUSTOMER'
    END AS customer_category
FROM Disbursements d
LEFT JOIN Repayments r ON d.customer_id = r.customer_id
GROUP BY d.customer_id, d.disb_date;
"""

# Execute query
customer_segments_df = pd.read_sql(query, conn)

# Display results
print("\nLoan Product Recommendations:")
print(customer_segments_df.head())



Loan Product Recommendations:
                                         customer_id  loan_amount  \
0  000514554c34603e8a7551050e988732cf11a22de40fa6...         3500   
1  000514554c34603e8a7551050e988732cf11a22de40fa6...         3500   
2  000514554c34603e8a7551050e988732cf11a22de40fa6...         3500   
3  000514554c34603e8a7551050e988732cf11a22de40fa6...         3500   
4  000514554c34603e8a7551050e988732cf11a22de40fa6...         3500   

   total_repaid  outstanding_balance  on_time_payments  late_payments  \
0       31100.0             -27600.0                30             10   
1       31100.0             -27600.0                30             10   
2       31100.0             -27600.0                30             10   
3       31100.0             -27600.0                30             10   
4       31100.0             -27600.0                30             10   

  customer_category  
0    LOYAL CUSTOMER  
1    LOYAL CUSTOMER  
2    LOYAL CUSTOMER  
3    LOYAL CUSTOMER  
4    